In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os

os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
    obtain_path,
)

In [34]:
%run notebooks/Rumours/load_rumours.py

In [35]:
df_rumours.head()

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0


In [36]:
df_rumours[df_rumours["timeline_id"] == "0"]

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0


In [37]:
df_rumours[:20]

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0
5,5.249425e+17,0,2014-10-22 15:16:52,"The University of Ottawa, just a five minute w...",1
6,5.249432e+17,0,2014-10-22 15:19:36,@ldobsonhughes @HuffPostCanada God Bless Heath...,1
7,5.249434e+17,0,2014-10-22 15:20:36,My Alma Mater. God. RT @ldobsonhughes: The Uni...,1
8,5.249441e+17,0,2014-10-22 15:23:16,@ldobsonhughes @InklessPW is 2 km from nasty p...,1
9,5.250329e+17,0,2014-10-22 21:16:06,BREAKING: Michael Zehaf-Bebeau had been design...,2


In [8]:
sbert_embeddings = torch.rand((5568, 384))

In [47]:
df_rumours.loc[9:12, "timeline_id"] = "0"

In [48]:
df_rumours[:20]

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0
5,5.249425e+17,0,2014-10-22 15:16:52,"The University of Ottawa, just a five minute w...",1
6,5.249432e+17,0,2014-10-22 15:19:36,@ldobsonhughes @HuffPostCanada God Bless Heath...,1
7,5.249434e+17,0,2014-10-22 15:20:36,My Alma Mater. God. RT @ldobsonhughes: The Uni...,1
8,5.249441e+17,0,2014-10-22 15:23:16,@ldobsonhughes @InklessPW is 2 km from nasty p...,1
9,5.250329e+17,0,2014-10-22 21:16:06,BREAKING: Michael Zehaf-Bebeau had been design...,0


In [9]:
"10" < "2"

True

In [49]:
test_df = df_rumours.sort_values(by=["timeline_id", "datetime"])

In [50]:
test_df[:20]

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0
9,5.250329e+17,0,2014-10-22 21:16:06,BREAKING: Michael Zehaf-Bebeau had been design...,0
10,5.250335e+17,0,2014-10-22 21:18:38,@20committee @BlogsofWar How come we have to p...,0
11,5.250335e+17,0,2014-10-22 21:18:40,@20committee Pathetic.,0
12,5.250336e+17,0,2014-10-22 21:19:09,@20committee @kshaidle maybe we should start r...,0
5,5.249425e+17,0,2014-10-22 15:16:52,"The University of Ottawa, just a five minute w...",1


In [43]:
test_df.sort_index()[:20]

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0
5,5.249425e+17,0,2014-10-22 15:16:52,"The University of Ottawa, just a five minute w...",1
6,5.249432e+17,0,2014-10-22 15:19:36,@ldobsonhughes @HuffPostCanada God Bless Heath...,1
7,5.249434e+17,0,2014-10-22 15:20:36,My Alma Mater. God. RT @ldobsonhughes: The Uni...,1
8,5.249441e+17,0,2014-10-22 15:23:16,@ldobsonhughes @InklessPW is 2 km from nasty p...,1
9,5.250329e+17,0,2014-10-22 21:16:06,BREAKING: Michael Zehaf-Bebeau had been design...,0


In [51]:
import nlpsig

test = nlpsig.PrepareData(
    original_df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...


In [52]:
path = test.pad(
    pad_by="history",
    embeddings="full",
    include_current_embedding=True,
    k=20,
    method="k_last",
)

[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

In [53]:
df_rumours[df_rumours["timeline_id"] == "0"]

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0
9,5.250329e+17,0,2014-10-22 21:16:06,BREAKING: Michael Zehaf-Bebeau had been design...,0
10,5.250335e+17,0,2014-10-22 21:18:38,@20committee @BlogsofWar How come we have to p...,0
11,5.250335e+17,0,2014-10-22 21:18:40,@20committee Pathetic.,0
12,5.250336e+17,0,2014-10-22 21:19:09,@20committee @kshaidle maybe we should start r...,0


In [60]:
test.df[test.df["timeline_id"] == "0"]

,index,id,label,datetime,text,timeline_id,e1,e2,e3,e4,...,e378,e379,e380,e381,e382,e383,e384,time_encoding,time_diff,timeline_index
0,0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,0.103396,0.427003,0.337202,0.427527,...,0.581833,0.792422,0.746922,0.178756,0.009332,0.177340,0.659595,2014.807584,0.000000,1
1,1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,0.892076,0.089191,0.010811,0.856021,...,0.699942,0.450611,0.599732,0.889609,0.789546,0.147721,0.196786,2014.807587,1.583333,2
2,2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,0.980686,0.805797,0.015934,0.167413,...,0.606664,0.123603,0.969136,0.644522,0.129121,0.469234,0.056136,2014.807589,0.800000,3
3,3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,0.358048,0.077240,0.503205,0.930801,...,0.661198,0.384447,0.886222,0.377348,0.084453,0.352526,0.935148,2014.807604,7.716667,4
4,4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,0.830996,0.744585,0.003194,0.763683,...,0.955724,0.447827,0.267471,0.498509,0.728802,0.820881,0.913358,2014.807688,44.200000,5
9,9,5.250329e+17,0,2014-10-22 21:16:06,BREAKING: Michael Zehaf-Bebeau had been design...,0,0.224529,0.722816,0.684795,0.805668,...,0.127077,0.449611,0.771965,0.540424,0.456456,0.278286,0.347250,2014.807907,115.416667,6
10,10,5.250335e+17,0,2014-10-22 21:18:38,@20committee @BlogsofWar How come we have to p...,0,0.269235,0.199696,0.070456,0.535430,...,0.190390,0.040539,0.066287,0.446543,0.436746,0.523888,0.357706,2014.807912,2.533333,7
11,11,5.250335e+17,0,2014-10-22 21:18:40,@20committee Pathetic.,0,0.547308,0.926500,0.346822,0.326619,...,0.540144,0.970637,0.948646,0.816785,0.132705,0.887803,0.738531,2014.807912,0.033333,8
12,12,5.250336e+17,0,2014-10-22 21:19:09,@20committee @kshaidle maybe we should start r...,0,0.525070,0.446079,0.009356,0.989078,...,0.844603,0.043178,0.343509,0.657212,0.537666,0.701899,0.490529,2014.807913,0.483333,9


In [56]:
path = path[:, :, :-2].astype("float")

In [57]:
path.shape

(5568, 20, 384)

In [59]:
path[5]

array([[0.07408142, 0.4766283 , 0.59138155, ..., 0.37458932, 0.8547917 ,
        0.01131529],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [65]:
path[13, :, :6]

array([[0.00523573, 0.12425941, 0.33464688, 0.86297834, 0.56450951,
        0.188586  ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0

## Using obtain_path

In [18]:
x_data = obtain_path(
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    k=20,
)

x_data.shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

(5568, 20, 384)

In [23]:
x_data[36]

array([[0.09438896, 0.40236121, 0.0424611 , ..., 0.75273949, 0.33756298,
        0.37205309],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [19]:
x_data[5, :, :6]

array([[0.76063091, 0.12631279, 0.26886719, 0.49926507, 0.94463491,
        0.63244152],
       [0.08725637, 0.46627671, 0.82576293, 0.41641968, 0.97817028,
        0.13840127],
       [0.64603299, 0.82171351, 0.1348384 , 0.05898112, 0.61936516,
        0.15020376],
       [0.50642848, 0.23047441, 0.006522  , 0.94399917, 0.31512761,
        0.20460558],
       [0.90076858, 0.30738407, 0.29393059, 0.67265528, 0.21803653,
        0.95985603],
       [0.71429211, 0.5487048 , 0.39909637, 0.46831083, 0.92386812,
        0.70548308],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0